In [1]:
from datetime import datetime, time, timedelta

import pandas as pd

from ib_async.ib import IB
from ib_async.contract import Stock
from ib_async import util

from tqdm.auto import tqdm

/Users/akseljoonas/.virtualenvs/news-sentiment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Connect IB


In [2]:
util.startLoop()
ib = IB()
ib.connect("127.0.0.1", 4001, clientId=1)

<IB connected to 127.0.0.1:4001 clientId=1>

Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfarm; secdefnj. The following farms are not connected: ushmds.
Error 162, reqId 33: Historical Market Data Service error message:No market data permissions for ISLAND STK, contract: Stock(conId=660300635, symbol='NEXI', exchange='SMART', primaryExchange='PINK', currency='USD', localSymbol='NEXI', tradingClass='LIMITED')
Error 162, reqId 34: Historical Market Data Service error message:No market data permissions for ISLAND STK, contract: Stock(conId=660300635, symbol='NEXI', exchange='SMART', primaryExchange='PINK', currency='USD', localSymbol='NEXI', tradingClass='LIMITED')
Error 162, reqId 38: Historical Market Data Service error message:No market data permissions for ISLAND STK, contract: Stock(conId=660300635, symbol='NEXI', exchange='SMART', primaryExchange='PINK', currency='USD', localSymbol='NEXI', tradingClass='LIMITED')
Error 162, reqId 

In [3]:
news_df = pd.read_csv(
    "/Users/akseljoonas/Documents/news-sentiment/data/raw/news_validated-new.csv"
)

### Import prices

In [4]:
def get_day_price(ticker, start_time):
    contract = Stock(ticker, "SMART", "USD")
    if not ib.qualifyContracts(contract):
        return pd.DataFrame()

    date = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S%z").date()
    end_of_day = datetime.combine(date, time(23, 59, 59))

    bar_data = ib.reqHistoricalData(
        contract,
        endDateTime=end_of_day.strftime("%Y%m%d %H:%M:%S"),
        durationStr="2 D",
        barSizeSetting="20 mins",
        whatToShow="TRADES",
        useRTH=False,
    )
    bar_data = util.df(bar_data)

    if bar_data is None or bar_data.empty:
        end_of_day = end_of_day + timedelta(days=1)
        bar_data = ib.reqHistoricalData(
            contract,
            endDateTime=end_of_day.strftime("%Y%m%d %H:%M:%S"),
            durationStr="3 D",
            barSizeSetting="20 mins",
            whatToShow="TRADES",
            useRTH=False,
        )
        bar_data = util.df(bar_data)
        if bar_data is None or bar_data.empty:
            bar_data = pd.DataFrame()

    return bar_data

In [5]:
import yfinance as yf


def get_yf_data(ticker, start_date):
    try:
        stock = yf.Ticker(ticker)
    except Exception:
        return None, None
    info = stock.info
    if info is None:
        return None, None

    try:
        float_shares = info["sharesOutstanding"]
    except KeyError:
        float_shares = None

    try:
        short_interest = info["dateShortInterest"]
    except KeyError:
        short_interest = None

    return float_shares, short_interest

In [6]:
def get_avg_volume(ticker, start_time):
    contract = Stock(ticker, "SMART", "USD")

    bar_data = ib.reqHistoricalData(
        contract,
        endDateTime=start_time.strftime("%Y%m%d %H:%M:%S"),
        durationStr="10 D",
        barSizeSetting="1 day",
        whatToShow="TRADES",
        useRTH=False,
    )
    
    if not bar_data:
        return 0
    
    bar_data = util.df(bar_data)
    return int(bar_data["volume"].mean())

In [7]:
def get_ibb_trend(start_time):
    # Calculate the start of the week

    contract = Stock("IBB", "SMART", "USD")
    bar_data = ib.reqHistoricalData(
        contract,
        endDateTime=start_time.strftime("%Y%m%d %H:%M:%S"),
        durationStr="10 D",
        barSizeSetting="1 day",
        whatToShow="TRADES",
        useRTH=False,
    )
    bar_data = util.df(bar_data)

    if bar_data.empty:
        return None  # No data available

    # Calculate the trend
    first_price = bar_data.iloc[0]["close"]
    last_price = bar_data.iloc[-1]["close"]

    # Return 1 if trend is positive, 0 otherwise
    return 1 if last_price > first_price else 0

In [8]:
def get_times(published_time):
    tzinfo = published_time.tzinfo
    # Define market times in EST
    premarket_start = time(0, 0, tzinfo=tzinfo)
    premarket_end = time(9, 30, tzinfo=tzinfo)
    market_start = time(9, 30, tzinfo=tzinfo)
    market_end = time(16, 0, tzinfo=tzinfo)
    after_hours_start = time(16, 0, tzinfo=tzinfo)
    after_hours_end = time(23, 59, 59, tzinfo=tzinfo)


    # Get the date component of the published_time
    prev_date = published_time - timedelta(days=1)
    next_date = published_time + timedelta(days=1)

    time_tz = published_time.time().replace(tzinfo=published_time.tzinfo)

    if premarket_start <= time_tz < premarket_end:
        start_time = pd.to_datetime(datetime.combine(prev_date, market_end))
        end_time = pd.to_datetime(datetime.combine(published_time, market_start))
    elif market_start <= time_tz < market_end:
        start_time = published_time
        end_time = published_time
    elif after_hours_start <= time_tz < after_hours_end:
        start_time = pd.to_datetime(datetime.combine(published_time, market_end))
        end_time = pd.to_datetime(datetime.combine(next_date, market_start))
    else:
        raise ValueError(
            f"Invalid time period for trading hours {published_time}, time_tz {time_tz}"
        )

    return start_time, end_time

In [9]:
import yfinance as yf


def get_market_cap(ticker: str) -> float | None:

    try:
        stock = yf.Ticker(ticker)
        market_cap = stock.info["marketCap"]
        # Convert to millions and format with 2 decimal places
        market_cap_millions = round(market_cap / 1_000_000, 2)
        return market_cap_millions
    except Exception:   
        return None



In [10]:
def format_bars(df, target_time, ticker):
    target_time = pd.to_datetime(target_time)

    market_start_time, market_end_time = get_times(target_time)

    start_time = market_start_time - timedelta(minutes=1)
    market_end_time = market_end_time + timedelta(hours=8)
    end_time = target_time + timedelta(hours=8)

    # day_df = df
    # daily_volume = day_df["volume"].sum()

    # Get the time needed
    df = df[(df["date"] >= start_time) & (df["date"] <= market_end_time)]

    if df.empty:
        return {
            "open": None,
            "high": None,
            "low": None,
            "close": None,
            "average": None,
            "volume": None,
            "trade_count": None,
            "vwap": None,
            # "relative_volume_daily": None,
            # "relative_volume_5m": None,
            # "float_rotation": None,
            # "short_interest": None,
            # "index_trend": None,
            "price_move": None,
            "labels": None,
            '5_labels': None,
            'market_cap': None,
            'buy_in_price': None,
        }

    # float_shares, short_interest = get_yf_data(ticker, start_time)
    # avg_daily_volume = get_avg_volume(ticker, start_time)
    # index_trend = get_ibb_trend(start_time)

    first_close = df["close"].iloc[0]
    first_open = df["open"].iloc[0]
    last_close = df.loc[df["date"].sub(end_time).abs().idxmin(), "close"]
    max_high = df["high"].max()
    min_low = df["low"].min()
    average = df["average"].mean()
    sum_volume = df["volume"].sum()
    sum_trade_count = df["barCount"].sum()
    buy_in_price = df.loc[df["date"] >= market_start_time, "open"].iloc[0]

    # relative_volume_daily = (
    #     daily_volume - avg_daily_volume
    # ) / avg_daily_volume + 0.00000000000000001
    # prev_5m_volume = day_df[
    #     (day_df["date"] >= target_time - pd.Timedelta(minutes=6))
    #     & (day_df["date"] <= start_time)
    # ]["volume"].mean()
    # relative_volume_5m = (
    #     sum_volume - prev_5m_volume
    # ) / prev_5m_volume + 0.00000000000000001
    # float_rotation = daily_volume / float_shares

    price_move = (last_close - first_close) / first_close

    market_cap = get_market_cap(ticker)

    if market_cap is None:
        market_cap = 0

    if market_cap > 10000:
        threshold = 0.01
    elif market_cap > 2000:
        threshold = 0.02
    elif market_cap > 250:
        threshold = 0.03
    else:
        threshold = 0.04

    if price_move > threshold:
        labels = 2
    elif price_move < -threshold:
        labels = 0
    else:
        labels = 1
        
    if price_move > threshold * 2:
        five_labels = 4
    elif price_move > threshold:
        five_labels = 3
    elif price_move < -threshold * 2:
        five_labels = 0
    elif price_move < -threshold:
        five_labels = 1
    else:
        five_labels = 2

    pv = df["average"] * df["volume"]
    cumulative_pv = sum(pv)
    aggregated_vwap = cumulative_pv / sum_volume

    return {
        "open": first_open,
        "high": max_high,
        "low": min_low,
        "close": last_close,
        "average": average,
        "volume": sum_volume,
        "trade_count": sum_trade_count,
        "vwap": aggregated_vwap,
        # "relative_volume_daily": relative_volume_daily,
        # "relative_volume_5m": relative_volume_5m,
        # "float_rotation": float_rotation,
        # "short_interest": short_interest,
        # "index_trend": index_trend,
        'market_cap': market_cap,
        'buy_in_price': buy_in_price,
        "price_move": price_move,
        '5_labels': five_labels,
        "labels": labels,
    }

In [11]:
import math


def add_stock_data(df):
    print(len(df))
    for index, row in tqdm(df.iterrows(), total=len(df)):
        # if index < 4000 or not math.isnan(row["labels"]):
        #     continue
        ticker = row["ticker"]

        start_time = row["published_eastern"]

        day_df = get_day_price(ticker, start_time)

        if not day_df.empty:
            print(f"{index} {ticker}")
            stock_info = format_bars(day_df, start_time, ticker)

            # Update row
            for col in stock_info.keys():
                df.loc[index, col] = stock_info[col]

        if index % 100 == 0:
            df.to_csv(
                "/Users/akseljoonas/Documents/news-sentiment/data/processed/news+prices-new-2.csv",
                index=False,
            )

    return df

In [12]:
import os


if os.path.exists(
    "/Users/akseljoonas/Documents/news-sentiment/data/processed/news+prices-new-2.csv"
):
    print("File exists")
    news_df = pd.read_csv(
        "/Users/akseljoonas/Documents/news-sentiment/data/processed/news+prices-new-2.csv"
    )


stock_df = add_stock_data(news_df)

stock_df.dropna(subset=["labels"], axis=0, inplace=True)
stock_df["topic"] = (
    stock_df["topic"]
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("/", "_")
    .str.replace("&", "")
    .str.replace("'", "")
)

4451


  0%|          | 0/4451 [00:00<?, ?it/s]

0 GRFS


  0%|          | 2/4451 [00:01<1:02:35,  1.18it/s]

1 GRFS


...

4448 AEON


100%|█████████▉| 4450/4451 [2:48:15<00:04,  4.93s/it]

4449 RENB


100%|██████████| 4451/4451 [2:48:27<00:00,  2.27s/it]

4450 VALN


In [13]:
stock_df.describe()

,open,high,low,close,average,volume,trade_count,vwap,market_cap,buy_in_price,price_move,5_labels,labels
count,4086.000000,4086.000000,4086.000000,4086.000000,4086.000000,4.086000e+03,4086.000000,4027.000000,4086.000000,4086.000000,4086.000000,4086.000000,4086.000000
mean,29.161416,31.679178,26.689382,28.144045,28.570022,1.703897e+06,4775.625551,29.000934,1451.204758,29.160166,0.004314,2.012482,1.005629
std,294.918568,324.657087,249.888300,267.104167,275.623642,8.469852e+06,21589.680384,278.154466,4261.260376,294.918379,0.117912,0.907788,0.596372
min,0.006400,0.006700,0.006100,0.006500,0.006329,0.000000e+00,0.000000,0.006289,0.000000,0.006400,-0.634228,0.000000,0.000000
25%,1.880000,2.070000,1.780000,1.886250,1.903080,5.837025e+04,222.000000,1.886394,48.570000,1.880000,-0.021431,2.000000,1.000000
50%,5.100000,5.473500,4.840000,5.100000,5.090034,2.720505e+05,1023.500000,5.053486,150.180000,5.100000,0.000000,2.000000,1.000000
75%,12.360000,12.960000,11.930000,12.400000,12.352134,8.726805e+05,2991.500000,12.563691,730.850000,12.360000,0.021046,2.000000,1.000000
max,10109.356000,10109.356000,6329.148100,6696.368300,7330.502747,1.940782e+08,481812.000000,7529.906579,29452.060000,10109.356000,2.433567,4.000000,2.000000


In [14]:
stock_df.to_csv(
    "/Users/akseljoonas/Documents/news-sentiment/data/processed/news+prices-new-2.csv",
    index=False,
)